In [1]:
import requests
from bs4 import BeautifulSoup
import json
import re
import pandas as pd
import time
pd.set_option('display.max_columns',None)
# pd.set_option('max_colwidth' , None)

* 'https://www.daraz.pk/womens-lawn/?page=1&spm=a2a0e.home.cate_4.1.35e34076QUW0yk

In [2]:
def get_html(url):
    try:
        headers={'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win 64 ; x64) Apple WeKit /537.36(KHTML , like Gecko) Chrome/80.0.3987.162 Safari/537.36'}
        res=requests.get(url,headers=headers)
        code = res.status_code
        return BeautifulSoup(res.text,'html.parser')
    except Exception as e:
        print(e)

In [3]:
def to_string(soup):
    json_data = soup.find_all('script')[3].text
    pattern= re.compile('(.*?)\{')
    num = pattern.match(json_data).span()[1]
    data=json.loads(json_data[num-1:])
    return data

In [4]:
def get_product_detail(productItems:list,data:dict):
    result_map = map(lambda x: {x:[]},productItems)
    products = {key: value for item in result_map for key, value in item.items()}
    listItems = data['mods']['listItems']
    for item in listItems:
        for element in productItems:
            products[element].append(item.get(element,None))
           
    return products

In [5]:
pages = 25
products = ['brandName',
            'name',
            'description',
            'location',
            'ratingScore',
            'review',
            'price'
            ]
daraz_pk=pd.DataFrame(columns=products)

for page in range(1,pages+1):
    
    url = f'https://www.daraz.pk/womens-lawn/?page={page}&spm=a2a0e.home.cate_4.1.35e34076QUW0yk'
    soup = get_html(url)
    data= to_string(soup)
    data=get_product_detail(products,data)
    daraz_pk=pd.concat([daraz_pk,pd.DataFrame(data)],ignore_index=True)    

In [6]:
daraz_pk.shape

(1000, 7)

In [7]:
daraz_pk.head()

,brandName,name,description,location,ratingScore,review,price
0,Libaas Hub,Libaas Hub Katan Silk 2Piece Un-stitched Suit ...,"[Brand:Libaas Hub, Gender: Women, Katan Silk U...",Pakistan,4.1,472,490.00
1,Ghazi Fabric,Ghazi fabric Unstitched Linen 2 piece suit for...,[Premium quality fabric Unstitched Gold Printe...,Pakistan,4.6,353,1280.00
2,No Brand,Ladies Navy Blue Embroidered Shalwar Kameez (U...,"[Color &amp; Design: As Shown in Picture, High...",Pakistan,4.8,404,2499.00
3,No Brand,cultural day special Ajrak shirt Dupatta,[Ajrak 2pc dress],Pakistan,5,1,1800.00
4,No Brand,Ajrak Chiffon 2pc,[Ajrak Chiffon Shirt &amp; Chiffon Ajrak Dupa...,Pakistan,5,2,2250.00
